# Automated Machine learningの説明の確認

機械学習がますます普及するにつれて、モデルによる予測は私たちの社会の多くの側面に大きな影響を与えます。たとえば、機械学習モデルは、銀行が融資を許可するか、医師が治療の優先順位を決定する方法においてますます重要な要素になります。モデルを解釈および説明する能力はますます重要になっているため、機械学習モデルによって行われた予測の理論的根拠を説明および正当化し、モデル内の不注意なバイアスを特定できます。

Automated Machine learningを使用してモデルをトレーニングする場合、各機能（前処理変換によって生成された工学的機能を含む）がラベル予測に影響を与える程度を定量化する変数の影響度の説明を生成するオプションがあります。このラボでは、自動化された機械学習実験によって生成された説明を調べます。


## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Automated Machine learning実験を実行する

このラボの時間を短縮するために、ローカルコンピューティングで3回の反復のみでAutomated Machine learning実験を実行します。

**model_explainability**設定オプションが**True**に設定されていることに注意してください。

In [ ]:
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

# Load the data
train_data = pd.read_csv('data/diabetes.csv')

# Configure Auto ML
automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target='local',
                             training_data = train_data,
                             n_cross_validations = 2,
                             label_column_name='Diabetic',
                             iterations=3,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=3,
                             featurization='off',
                             model_explainability=True # Generate feature importance!
                             )

# Run the Auto ML experiment
print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion()

## 変数の影響度を表示

上記のウィジェットで実験が完了したら、最良の結果が得られた実行をクリックして詳細を表示します。次に、視覚化の下部までスクロールして、相対的な変数の影響度を確認します。

**ExplanationClient**クラスを使用して、実験で作成された最適なモデルの変数の影響度を表示することもできます:

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# Get the best model (2nd item in outputs)
best_run, fitted_model = automl_run.get_output()

# Get the feature explanations
client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

## Azure Machine Learning Studioでモデルの説明を表示する

実験の実行が完了したら、ウィジェットのリンクをクリックして、Azure Machine Learning Studioで実行を確認し、**説明**タブを表示します。次に：

1. Automated Machine learningの実行によって作成された*explainer*を選択します。
2. 全体的なグローバル変数の影響度を示す**Global Importance**チャートを表示します。
3. *swarm*、*violin*、または*box*プロットのテストデータの各データポイントを示す**Summary Importance**チャートを表示します。
4. 個々のポイントを選択して、選択したデータポイントの個々の予測の**Local feature Importance**を表示します。

## 自動設計された変数の影響度を表示

Automated Machine Learningには、データの前処理を試みるオプションが含まれます。これは、多くの場合、*機能強化*を実行して、モデルをトレーニングする新しい派生機能を作成します。そのオプションを有効にして、自動化された機械学習実験を再実行しましょう。

In [ ]:
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

# Load the data
train_data = pd.read_csv('data/diabetes.csv')

# Configure Auto ML
automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target='local',
                             training_data = train_data,
                             n_cross_validations = 2,
                             label_column_name='Diabetic',
                             iterations=3,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=3,
                             featurization='auto', # Enable feature engineering
                             model_explainability=True # Generate feature importance
                             )

# Run the Auto ML experiment
print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion()

機能は、[Scikit-Learn変換パイプライン](https://scikit-learn.org/stable/modules/compose.html#combining-estimators)を使用して実現されます（Azure Machine Learningパイプラインと混同しないでください！）。これらは、推論の前にデータを変換する手順を含むモデルを生成します。

Run the following code to view the steps in the model pipeline, and the importance of the engineered features.

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# Get the best model (2nd item in outputs)
best_run, fitted_model = automl_run.get_output()

print('Model pipeline steps:')
for step in fitted_model.named_steps:
    print('-',step)
    
# Get the feature explanations
client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=False)
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('\nFeature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

> **詳しくは**: Automated Machine Learningの詳細については、[Azure MLドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train)を参照してください。